<a href="https://colab.research.google.com/github/Nagendra061192/Open-Source-Projects-GenAI-/blob/main/Llama2_on_CPU_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Nagendra061192/Llama-2-on-CPU-Machine.git

Cloning into 'Llama-2-on-CPU-Machine'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (5/5), done.


In [2]:
import os
from pathlib import Path
import logging
!pip install langchain
!pip install CTransformers
!pip install pypdf
!pip install sentence-transformers
!pip install faiss-cpu



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_n

In [3]:
logging.basicConfig(level=logging.INFO, format='[%(asctime)s]:%(message)s:')

In [32]:
list_of_files = [
    "src/_init_.py",
    "src/run_local.py",
    "src/helper.py",
    "model/instruction.txt",
    "requirements.txt",
    "setup.py",
    "main.py",
    "research/trials.ipynb",
    ]




for filepath in list_of_files:
  filepath=Path(filepath)
  filedir,filename = os.path.split(filepath)



  if filedir !="":
    os.makedirs(filedir, exist_ok=True)
    logging.info(f"creating directory; {filedir} for the file:{filename}")

  if (not os.path.exists(filepath)) or (os.path.getsize(filepath)==0):
    with open (filepath,"w") as f:
      pass
      logging.info(f"creating empty file: {filepath}")

  else:
      logging.info(f"{filename} is already exists")

In [5]:
!pip install -r requirements.txt

In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3


In [7]:
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.llms import CTransformers


B_INST,E_INST= "[INST]","[/INST]"
B_SYS, E_SYS= "<<SYS>>\n","\n<</SYS>>\n\n"

In [8]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of
answering something not correct. If you don't know the answer to a question,
please don't share false information."""


In [9]:
instruction="convert the following text from english to french:\n\n {text} "

In [10]:
SYSTEM_PROMPT= B_SYS+DEFAULT_SYSTEM_PROMPT+E_SYS
template= B_INST+SYSTEM_PROMPT+instruction+E_INST

In [11]:
prompt=PromptTemplate(template=template, input_variables=["text"])

In [12]:
llm=CTransformers(model= "TheBloke/Llama-2-7B-Chat-GGUF",
                  model_type="llama",
                  config={"max_new_tokens":128,
                          "temperature":0.01}
                  )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.Q2_K.gguf:   0%|          | 0.00/2.83G [00:00<?, ?B/s]

In [13]:
LLM_Chain=LLMChain(prompt=prompt, llm=llm)

In [14]:
print(LLM_Chain.run("How are you?"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


  Bonjour! Comment vas-tu? (How are you?)


Translation from English to Hindi using Custom prompt



In [15]:
CUSTOM_SYSTEM_PROMPT="""\
You are an advanced assistant that provides translation from English to Hindi language."""

In [16]:
instruction="convert the following text from English to Hindi:\n\n {text} "

In [17]:
SYSTEM_PROMPT= B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS
template= B_INST+SYSTEM_PROMPT+instruction+E_INST

In [18]:
prompt=PromptTemplate(template=template, input_variables=["text"])

In [19]:
LLM_Chain=LLMChain(prompt=prompt, llm=llm)

In [20]:
print(LLM_Chain.run("How are you?"))

  Sure, I'd be happy to help! Here is the translation of "How are you?" in Hindi:

कैसे हैं आप (Kaise hai aap)

I hope that helps! Let me know if you have any other questions.


Text Summarization using Custom Prompt


In [21]:
CUSTOM_SYSTEM_PROMPT="""\
You are an advanced assistant that provides summarization given any book name."""

In [22]:
instruction="Provide a summary of the book given book name:\n\n {text} "

In [23]:
SYSTEM_PROMPT= B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS
template= B_INST+SYSTEM_PROMPT+instruction+E_INST

In [24]:
prompt=PromptTemplate(template=template, input_variables=["text"])

In [25]:

LLM_Chain=LLMChain(prompt=prompt, llm=llm)

In [27]:
print(LLM_Chain.run("Harry Potter"))

  Of course! Here's a summary of the book "Harry Potter" by J.K. Rowling:

The story follows the adventures of a young wizard named Harry Potter, who discovers he is the son of two powerful wizards who were killed by the dark wizard, Lord Voldemort. Harry learns that he has been chosen to attend Hogwarts School of Witchcraft and Wizardry, where he makes friends with Ron Weasley and Hermione Granger.

Throughout the series, Harry and his friends must contend with the challeng


RAG using PDF


In [41]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from src.helper import *
from flask import Flask,render_template,jsonify,request

In [80]:
app = Flask(__name__)

In [54]:
loader=DirectoryLoader("/Data/",
                       glob="*.pdf",
                       loader_cls=PyPDFLoader)

In [55]:
documnets=loader.load()

In [56]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,
                                             chunk_overlap=20)
text_chunks=text_splitter.split_documents(documnets)

In [57]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-l6-v2",
                                 model_kwargs={"device":"cpu"})

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [58]:
vector_store=FAISS.from_documents(text_chunks,embeddings)

In [59]:
llm=CTransformers(model= "TheBloke/Llama-2-7B-Chat-GGUF",
                  model_type="llama",
                  config={"max_new_tokens":128,
                          "temperature":0.01}
                  )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [60]:
template="""Use the following peices of information to answer the user's question.
if you dont know the answer just say you dont know,dont try to make up an answer.
Context:{context}
Question:{question}
Only return the helpful answer below and nothing else
Helpful answer
"""

In [61]:
qa_prompt=PromptTemplate(template=template, input_variables=["context","question"])

In [87]:
chain=RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=vector_store.as_retriever(search_kwargs={"k":2}),
                                  return_source_documents=False,
                                  chain_type_kwargs={"prompt":qa_prompt})

In [88]:
user_input="Tell me about Discussion of Results of the paper"

In [86]:
result = chain({"query": user_input})
print(f"answer: {result['result']}")

answer: The authors compare YOLO against the previous work described above on PASCAL VOC 2007, and VOC 2012 as well as out of domain art dataset. They use two performance metrics: mAP score and FPS.


In [81]:
@app.route('/', methods=["GET", "POST"])
def index():
    return render_template('index.html')

In [82]:
@app.route('/chatbot', methods=["GET", "POST"])
def chatbotResponse():
    if request.method == 'POST':
        user_input = request.form['question']
        print(user_input)

        # Replace the following line with your actual chatbot logic
        result = {'result': 'Hello, world!'}

        print(f"Answer:{result['result']}")

    return jsonify({"response": str(result['result']) })

In [85]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port= 8081,debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8081
 * Running on http://172.28.0.12:8081
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
